In [1]:
import numpy as np
import pandas as pd
from tensorflow import keras

In [2]:
train = pd.read_json('train.json', lines=True)

In [3]:
tmp1 = np.zeros((100,4))
tmp2 = np.zeros((100,16))
tmp3 = np.zeros((100,7))

In [4]:
out1 = np.zeros((100,1))
out2 = np.zeros((100,1))
out3 = np.zeros((100,1))

## Modele Sequentiel

Il y a ici un input de dimension 27 pour la sequence (4 colonnes), les appariement (16) et les types de loops (7)

In [5]:
model_simple = keras.Sequential()

model_simple.add(keras.layers.Dense(10, input_dim=27, activation=('relu')))
model_simple.add(keras.layers.Dense(8, activation='relu'))
model_simple.add(keras.layers.Dense(4, activation='relu'))
model_simple.add(keras.layers.Dropout(rate=0.4))

#Est-ce que le out put c'est 3 valuers distinctes, ou 1 valeurs mais qui existe dans 3 channels?
model_simple.add(keras.layers.Dense(3, activation='linear'))

# mean_squared_error car on a affair a une regression
model_simple.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])
model_simple.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 10)                280       
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 88        
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 36        
_________________________________________________________________
dropout (Dropout)            (None, 4)                 0         
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 15        
Total params: 419
Trainable params: 419
Non-trainable params: 0
_________________________________________________________________


## Modeles utilisant le 'functional API'

In [6]:

inputs = keras.layers.Input(shape=(68,27,)) #Si on considere juste la dimensionalite des entrees,
                                            #en considerant les 68 positions et les 27 colonnes
dense_1 = keras.layers.Dense(10)(inputs)
dense_2 = keras.layers.Dense(10)(inputs)
dense_1 = keras.layers.Dense(3)(dense_1)
dense_2 = keras.layers.Dense(3)(dense_2)

outputs = keras.layers.average([dense_1, dense_2]) #moyenne des 2 couches fournies

model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer='adam', loss='mse', metrics=['mse'])
model.summary()


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 68, 27)       0                                            
__________________________________________________________________________________________________
dense_4 (Dense)                 (None, 68, 10)       280         input_1[0][0]                    
__________________________________________________________________________________________________
dense_5 (Dense)                 (None, 68, 10)       280         input_1[0][0]                    
__________________________________________________________________________________________________
dense_6 (Dense)                 (None, 68, 3)        33          dense_4[0][0]                    
__________________________________________________________________________________________________
dense_7 (D

In [7]:
seq_input = keras.layers.Input(shape=(4,), name="seq")      #Input shape a revoir
pair_input = keras.layers.Input(shape=(16,), name="pair")  
loop_input = keras.layers.Input(shape=(7,), name="loop")  

seq_features = keras.layers.Dense(20)(seq_input)
pair_features = keras.layers.Dense(20)(pair_input)
loop_features = keras.layers.Dense(20)(loop_input)


# Merge les features
x = keras.layers.concatenate([seq_features, pair_features, loop_features])


first_pred = keras.layers.Dense(1, name="reactivity")(x) #regression pour "reactivity"
second_pred = keras.layers.Dense(1, name="ph")(x)  #regression pour "ph"
third_pred = keras.layers.Dense(1, name="tmp")(x)


model = keras.Model(
    inputs=[seq_input, pair_input, loop_input],
    outputs=[first_pred, second_pred, third_pred],
)
model.compile(optimizer='adam', loss='mse', metrics=['mse'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
seq (InputLayer)                (None, 4)            0                                            
__________________________________________________________________________________________________
pair (InputLayer)               (None, 16)           0                                            
__________________________________________________________________________________________________
loop (InputLayer)               (None, 7)            0                                            
__________________________________________________________________________________________________
dense_8 (Dense)                 (None, 20)           100         seq[0][0]                        
__________________________________________________________________________________________________
dense_9 (D

In [9]:
model.fit([tmp1,tmp2,tmp3],[out1,out2,out3],epochs=20,batch_size=10,validation_split=0.1)

Train on 90 samples, validate on 10 samples
Epoch 1/20
90/90 [==============================] - ETA: 0s - loss: 0.0000e+00 - reactivity_loss: 0.0000e+00 - ph_loss: 0.0000e+00 - tmp_loss: 0.0000e+00 - reactivity_mean_squared_error: 0.0000e+00 - ph_mean_squared_error: 0.0000e+00 - tmp_mean_squared_error: 0.0000e+00 - 4s 46ms/step - loss: 0.0000e+00 - reactivity_loss: 0.0000e+00 - ph_loss: 0.0000e+00 - tmp_loss: 0.0000e+00 - reactivity_mean_squared_error: 0.0000e+00 - ph_mean_squared_error: 0.0000e+00 - tmp_mean_squared_error: 0.0000e+00 - val_loss: 0.0000e+00 - val_reactivity_loss: 0.0000e+00 - val_ph_loss: 0.0000e+00 - val_tmp_loss: 0.0000e+00 - val_reactivity_mean_squared_error: 0.0000e+00 - val_ph_mean_squared_error: 0.0000e+00 - val_tmp_mean_squared_error: 0.0000e+00
Epoch 2/20
90/90 [==============================] - 0s 444us/step - loss: 0.0000e+00 - reactivity_loss: 0.0000e+00 - ph_loss: 0.0000e+00 - tmp_loss: 0.0000e+00 - reactivity_mean_squared_error: 0.0000e+00 - ph_mean_square

90/90 [==============================] - 0s 333us/step - loss: 0.0000e+00 - reactivity_loss: 0.0000e+00 - ph_loss: 0.0000e+00 - tmp_loss: 0.0000e+00 - reactivity_mean_squared_error: 0.0000e+00 - ph_mean_squared_error: 0.0000e+00 - tmp_mean_squared_error: 0.0000e+00 - val_loss: 0.0000e+00 - val_reactivity_loss: 0.0000e+00 - val_ph_loss: 0.0000e+00 - val_tmp_loss: 0.0000e+00 - val_reactivity_mean_squared_error: 0.0000e+00 - val_ph_mean_squared_error: 0.0000e+00 - val_tmp_mean_squared_error: 0.0000e+00
Epoch 17/20
90/90 [==============================] - 0s 444us/step - loss: 0.0000e+00 - reactivity_loss: 0.0000e+00 - ph_loss: 0.0000e+00 - tmp_loss: 0.0000e+00 - reactivity_mean_squared_error: 0.0000e+00 - ph_mean_squared_error: 0.0000e+00 - tmp_mean_squared_error: 0.0000e+00 - val_loss: 0.0000e+00 - val_reactivity_loss: 0.0000e+00 - val_ph_loss: 0.0000e+00 - val_tmp_loss: 0.0000e+00 - val_reactivity_mean_squared_error: 0.0000e+00 - val_ph_mean_squared_error: 0.0000e+00 - val_tmp_mean_squa

## Modele avec recurrence simple